# Pruebas automatizadas datos humedad relativa - Clase

> Elaborado por Paola Álvarez, profesional contratista IDEAM, contrato 196 de 2024. Comentarios o inquietudes, remitir a *palvarez@ideam.gov.co* 

**Librerías**

In [1]:
import pandas as pd
import numpy as np
import os
import re
import logging
from functools import wraps
from functools import reduce

----

A continuación, se encuentran las pruebas de pre-validación de datos de EMA para verificar su capacidad de detección de datos

## Clase con métodos de aplicación de QC

In [3]:
# Configuración del logger para guardar en el directorio de archivos y sobrescribir cada vez
def setup_logger(log_file_path):
    logger = logging.getLogger('RawUnmodified_TS-10cm')
    logger.setLevel(logging.INFO)
    # Clear existing handlers to avoid duplicate logs
    if logger.hasHandlers():
        logger.handlers.clear()
    file_handler = logging.FileHandler(log_file_path, mode='a')
    file_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
    logger.addHandler(file_handler)
    return logger

In [77]:
class AutomatTS10cmEMA:
    
    def __init__(self, dir_files, chunk_size=540000):
        self.dir_files = dir_files
        self.ruta_archivos = os.listdir(dir_files)
        self.chunk_size = chunk_size
        self.last_rows = None
        self.current_file = None
        # Sección configuración de logs
        log_file_path = os.path.join(dir_files, 'QC_TS-10cm.log')
        self.logger = setup_logger(log_file_path)
        self.logger.info('Inicialización de PreValidTS10EMA en directorio: %s', dir_files)

    def process_freqs(self, chunk, archivo):
        '''Esta función procesa las frecuencias para abreviar variedad de métodos en adelante'''
        # Convertir la columna de fecha a datetime si aún no lo es
        if not pd.api.types.is_datetime64_any_dtype(chunk['Fecha']):
            chunk['Fecha'] = pd.to_datetime(chunk['Fecha'])
            
        # Cargar el archivo de frecuencias
        freqinst200b = pd.read_csv('EMATS10_LatLonEntFreq.csv', encoding='latin-1') #, sep=';')
    
        # Definir el diccionario de frecuencias y cantidades esperadas
        frecuencias = {
            'min': {'cant_esperd_h': 60, 'cant_esperd_d': 1440, 'cant_esperd_m': 43200, 
                  'cant_esperd_a': 518400, 'minutos': 1, 'shiftnum': 92, 'jumpnum':15},
            '5min': {'cant_esperd_h': 12, 'cant_esperd_d': 288, 'cant_esperd_m': 8640, 
                   'cant_esperd_a': 103680, 'minutos': 5, 'shiftnum': 27, 'jumpnum':20},
            '10min': {'cant_esperd_h': 6, 'cant_esperd_d': 144, 'cant_esperd_m': 4320, 
                    'cant_esperd_a': 51840, 'minutos': 10, 'shiftnum': 19, 'jumpnum':25},
            'h': {'cant_esperd_h': 1, 'cant_esperd_d': 24, 'cant_esperd_m': 720, 
                  'cant_esperd_a': 8640, 'shiftnum':12, 'jumpnum':35} ## Cambiar para adaptarse a directriz GGD
        }
    
        # Obtener el valor de la estación
        station_value = chunk['Station'].values[0]
        if pd.isna(station_value):
            print(f'La estación {station_value} no se encuentra en el análisis de frecuencias')
            return {'periodos': None, 'frecuencias': None}
        else:
            freqinst200b_station = freqinst200b.loc[freqinst200b['Station'] == station_value]
            if freqinst200b_station.empty:
                print(f"No se encontró la estación {station_value} en freqinst200b")
                return {'periodos': None, 'frecuencias': None}
            else:
                freq_inf_value = freqinst200b_station['FreqInf'].values[0]
    
            if pd.isna(freq_inf_value):
                try:
                    periodos = pd.infer_freq(chunk['Fecha'][-25:])
                    print(periodos)
                    if periodos is None:
                        print(f"Frecuencia inferida es None para el archivo {archivo}")
                        return {'periodos': None, 'frecuencias': None}
                except ValueError as e:
                    print(f'Error al inferir la frecuencia en el archivo {archivo}: {str(e)}')
                    return {'periodos': None, 'frecuencias': None}
            else:
                periodos = freq_inf_value
    
        if periodos is None:
            print(f"Periodo es None para el archivo {archivo}")
            return {'periodos': None, 'frecuencias': None}
    
        if periodos in frecuencias:
            return {'periodos': periodos, 'frecuencias': frecuencias[periodos]}
        else:
            print(f"Periodo {periodos} no es reconocido en el diccionario de frecuencias")
            return {'periodos': None, 'frecuencias': None}

    def p_transm(self, chunk, archivo):
        '''Esta prueba verifica si existe al menos el 70% de datos esperados por día y hora
        en la serie de datos; aquellos que no superen la prueba, son marcados como sospechosos'''
        # Se instancia el método 'process_freqs' para obtener las frecuencias 
        freq_info = self.process_freqs(chunk, archivo)
        if freq_info is None or freq_info['periodos'] is None or freq_info['frecuencias'] is None:
            print(f"No se pudo obtener información de frecuencia para el archivo {archivo}")
            return chunk
        
        periodos = freq_info['periodos']
        frecuencias = freq_info['frecuencias']

        # Obtener las cantidades esperadas de acuerdo a la frecuencia
        cant_esperd_h = frecuencias['cant_esperd_h']
        cant_esperd_d = frecuencias['cant_esperd_d']
    
        # Se establecen los aceptables
        cant_aceptab_hora = 0.7 * cant_esperd_h
        cant_aceptab_dia = 0.7 * cant_esperd_d
    
        # Agregar columna de etiquetas al dataframe original
        chunk['Estado'] = ''
        
        # Definir función para asignar etiquetas y llenar archivo log
        def asignar_etiqueta(row):
            if row['count'] < cant_aceptab_hora:
                filas_fallidas = chunk.loc[chunk['Fecha'].dt.floor('h') == row['Fecha'].floor('h')]
                chunk.loc[filas_fallidas.index, 'Estado'] = '0PSO0'
                # Registrar en el log las filas que fallaron
                for index, fila in filas_fallidas.iterrows():
                    self.logger.info('File %s - Row %s - failed hour p_transm: %s', archivo, index, fila['Fecha'])

        # Evaluar por cada grupo de datos por hora y asignar la etiqueta
        canthora = chunk.groupby(chunk['Fecha'].dt.floor('h')).size().reset_index(name='count')
        canthora.apply(asignar_etiqueta, axis=1)
        
        # Definir función para asignar etiquetas de acumulado diario y llenar archivo log
        def asignar_etiqueta_diaria(row):
            if row['count'] < cant_aceptab_dia:
                filas_fallidas_dia = chunk.loc[chunk['Fecha'].dt.floor('D') == row['Fecha'].floor('D')]
                chunk.loc[filas_fallidas_dia.index, 'Estado'] = '0PSO0'
                # Registrar en el log las filas que fallaron
                for index, fila in filas_fallidas_dia.iterrows():
                    self.logger.info('File %s - Row %s - failed day p_transm: %s', archivo, index, fila['Fecha'])

        # Evaluar por cada grupo de datos por día y asignar la etiqueta
        cantdia = chunk.groupby(chunk['Fecha'].dt.floor('D')).size().reset_index(name='count')
        cantdia.apply(asignar_etiqueta_diaria, axis=1)
        
        return chunk

    def p_estruc(self, chunk, archivo):
        '''Esta prueba verifica si los datos fueron transmitidos en horas y minutos exactos al ser el
        comportamiento esperado'''
        # Se crea la columna 'Estado' si no existe
        if 'Estado' not in chunk.columns:
            chunk['Estado'] = ''
        # Se instancia el método 'process_freqs' para obtener las frecuencias 
        freq_info = self.process_freqs(chunk, archivo)
        # Se verifica si 'freq_info' is None
        if freq_info is None or freq_info['periodos'] is None or freq_info['frecuencias'] is None:
            print(f"No se pudo obtener información de frecuencia para el archivo {archivo}")
            return chunk
        periodos =  freq_info['periodos']
        frecuencias = freq_info['frecuencias']

        # Generar la operación para observar si la estructura es exacta en minutos
        fecha = chunk['Fecha']
    
        # Se vectoriza la evaluación de la estructura por minuto para cada chunk:
        if periodos == 'min':
            mask_estr = fecha.dt.second != 0
        elif periodos == 'h':
            mask_estr = (fecha.dt.minute != 0) | (fecha.dt.second != 0)
        else:
            # Se obtiene num_para_modulo
            num_para_modulo = frecuencias['minutos']
            mask_estr = fecha.dt.minute % num_para_modulo != 0
    
        # Se registran los errores en el log
        if mask_estr.any():
           aligned_mask = mask_estr.reindex(chunk.index, fill_value=False)
           for index, row in chunk[aligned_mask].iterrows():
               self.logger.info('File %s - Row %s - failed time p_estruc: %s', archivo, index, row['Fecha'])
        else:
           self.logger.info('File: %s - No se encontraron fallos en p_estruc', archivo)
        
        chunk['Estado'] = chunk['Estado'].fillna('')
        # Lógica de etiquetado para 'Estado', sospechoso, '0PSO0'
        condicion_0PSO0 = mask_estr & (chunk['Estado']=='')
        chunk.loc[condicion_0PSO0, 'Estado'] = '0PSO0'
        mask_estr = mask_estr & ~condicion_0PSO0
        # 0PSO1
        condicion_0PSO1 = mask_estr & (chunk['Estado'] == '0PSO0')
        chunk.loc[condicion_0PSO1, 'Estado'] = '0PSO1'

        return chunk

    def p_limrig(self, chunk, archivo):
        '''Esta prueba verifica si los datos crudos se encuentran fuera del umbral físico inferior o superior'''
        # Se crea la columna 'Estado' si no existe
        if 'Estado' not in chunk.columns:
            chunk['Estado'] = ''
        # Se genera la columna de estado anterior
        chunk['Estado_Anterior'] = ''
        
        # Se establecen los umbrales físicos/rígidos a datos crudos en nuevas colummnas para vectorizar
        chunk['umbr_crud_inf'] = -15.0
        chunk['umbr_crud_sup'] = 40.0

        # Compara el dato con umbrales inferiores y superiores 
        mask_outbounds = (chunk['Valor'] < chunk['umbr_crud_inf']) | (chunk['Valor'] > chunk['umbr_crud_sup'])

        # Se registran los errores en el log
        if mask_outbounds.any():
            aligned_mask_lr = mask_outbounds.reindex(chunk.index, fill_value=False)
            for index, row in chunk[aligned_mask_lr].iterrows():
                self.logger.info('File %s - Row %s - failed val p_limrig: %s', archivo, index, row['Valor'])
        else:
            self.logger.info('File: %s - No se encontraron fallos en p_limrig', archivo)

        chunk['Estado'] = chunk['Estado'].fillna('')
        # Lógica de etiquetado para 'Estado_Anterior'
        condicion_0PSO0 = mask_outbounds & chunk['Estado'].isin(['0PSO0', '0PSO1'])
        chunk.loc[condicion_0PSO0, 'Estado_Anterior'] = chunk.loc[condicion_0PSO0, 'Estado']
        # Lógica de etiquetado para 'Estado'
        condicion_0PER0 = mask_outbounds & ((chunk['Estado']=='') | chunk['Estado'].isin(['0PSO0', '0PSO1']))
        chunk.loc[condicion_0PER0, 'Estado'] = '0PER0'
        
        # Se eliminan las columnas no deseadas
        if 'umbr_crud_inf' in chunk.columns:
            chunk.drop(columns=['umbr_crud_inf', 'umbr_crud_sup'], axis=1, inplace=True)
                
        return chunk

    def p_perst(self, chunk, archivo):
        '''Esta prueba detecta los datos que se repiten por más de cuatro horas consecutivas'''
        # Se crea la columna 'Estado' si no existe
        if 'Estado' not in chunk.columns:
            chunk['Estado'] = ''    
        # Se genera la columna 'Estado_anterior' si no existe
        if 'Estado_Anterior' not in chunk.columns:
            chunk['Estado_Anterior'] = ''

        # Verificar si el archivo ha cambiado
        if self.current_file != archivo:
            # Si el archivo cambió, resetea self.last_rows y actualiza self.current_file
            self.last_rows = None
            self.current_file = archivo
            
        # Guardar referencia a las filas originales del chunk actual
        original_chunk = chunk.copy()
            
        # Usar self.last_rows para concatenar con el chunk actual
        if self.last_rows is not None:
            chunk = pd.concat([self.last_rows, chunk])
            chunk.reset_index(drop=True)

        # Verificar y eliminar duplicados en el índice después de la concatenación
        if not chunk.index.is_unique:
            chunk = chunk.reset_index(drop=True)  # Crear un nuevo índice único

        # Crear una etiqueta temporal para saber qué filas provienen del chunk anterior
        if self.last_rows is not None:
            chunk['from_previous_chunk'] = [True] * len(self.last_rows) + [False] * len(original_chunk)
        else:
            chunk['from_previous_chunk'] = [False] * len(original_chunk)

        # Se crean máscaras para el intervalo del día con radiación solar que puede afectar la humedad
        mask_sunny = (chunk['Fecha'].dt.hour >= 5) & (chunk['Fecha'].dt.hour <= 19)
        # Se filtran los datos para esas horas
        mask_sun = chunk[mask_sunny]

        ## Se manejan las distintas frecuencias para verificar adecuadamente las persistencias
        # Se instancia el método 'process_freqs' para obtener las frecuencias 
        freq_info = self.process_freqs(chunk, archivo)
        # Se verifica si 'freq_info' is None
        if freq_info is None or freq_info['periodos'] is None or freq_info['frecuencias'] is None:
            print(f"No se pudo obtener información de frecuencia para el archivo {archivo}")
            return original_chunk
        
        # Se accede a las claves del diccionario
        periodos =  freq_info['periodos']
        # Se accede a los datos del diccionario
        frecuencias = freq_info['frecuencias']
        # Se asignan los shiftnums
        cantshifts = frecuencias['shiftnum']
        
        # Crear una lista de máscaras usando una lista por comprensión
        masks = [(mask_sun['Valor'] == mask_sun['Valor'].shift(i)) for i in range(1, cantshifts + 1)]

        # Combinar todas las máscaras usando reduce y operador &
        from functools import reduce
        mask_persdatos = reduce(lambda x, y: x & y, masks)
        
        #chunk.reset_index(drop=True)
        # Se registran los errores en el log
        if mask_persdatos.any():
            aligned_mask_pers = mask_persdatos.reindex(chunk.index, fill_value=False)
            for index, row in chunk[aligned_mask_pers].iterrows():
                self.logger.info('File %s - Row %s - failed val p_perst: %s', archivo, index, row['Valor'])
        else:
            self.logger.info('File: %s - No se encontraron fallos en p_perst', archivo)

        chunk['Estado'] = chunk['Estado'].fillna('')
        # Etiquetado de valores, se inicia con el Estado Anterior
        condicion_0PSO0 = mask_persdatos & chunk['Estado'].isin(['0PSO0', '0PSO1'])
        chunk.loc[condicion_0PSO0, 'Estado_Anterior'] = chunk.loc[condicion_0PSO0, 'Estado']

        # Lógica de etiquetado para 'Estado'
        condicion_0PER0 = mask_persdatos & ((chunk['Estado']=='') | chunk['Estado'].isin(['0PSO0', '0PSO1']))
        chunk.loc[condicion_0PER0, 'Estado'] = '0PER0'
        mask_persdatos = mask_persdatos & ~condicion_0PER0

        condicion_0PER1 = mask_persdatos & (chunk['Estado'] == '0PER0')
        chunk.loc[condicion_0PER1, 'Estado'] = '0PER1'

        # Eliminar solo las filas del chunk anterior que fueron evaluadas
        chunk = chunk[~chunk['from_previous_chunk']].copy()

        # Eliminar la columna temporal
        chunk.drop(columns=['from_previous_chunk'], inplace=True)

        # Actualizar las filas de self.last_rows para el próximo chunk
        self.last_rows = chunk.tail(cantshifts).copy()

        return chunk

    def p_jump(self, chunk, archivo):
        '''Esta prueba verifica si la variación entre valores consecutivos excede 45.0 %'''
        # Se crea la columna 'Estado' si no existe
        if 'Estado' not in chunk.columns:
            chunk['Estado'] = ''
        # Se genera la columna 'Estado_anterior' si no existe
        if 'Estado_Anterior' not in chunk.columns:
            chunk['Estado_Anterior'] = ''

        # Se hace una instancia del método de 'process_freqs' para obtener las frecuencias
        freq_info = self.process_freqs(chunk, archivo)
        if freq_info is None or freq_info['periodos'] is None or freq_info['frecuencias'] is None:
            print(f"No se pudo obtener información de frecuencia para el archivo {archivo}")
            return chunk

        periodos = freq_info['periodos']
        frecuencias = freq_info['frecuencias']

        # Asegurarse de que 'periodos' tenga un número antes de la unidad
        if periodos.isalpha():
            periodos = '1' + periodos
        
        # Ordenar el chunk por la columna 'Fecha'
        chunk = chunk.sort_values('Fecha').reset_index(drop=True)
        
        # Se genera filtro para no considerar datos ya catalogados como erróneos
        chunk['Estado'] = chunk['Estado'].fillna('')
        if chunk['Estado'].notna().all(): # Se verifica que no hayan valores nulos en tal columna
            chunk_jmp = chunk[~chunk['Estado'].str.startswith('0PER', na=False)].copy()
        else:
            # Si todos los valores son NaN, se copia el chunk
            chunk_jmp = chunk.copy()
        
        # Crear una columna de diferencia temporal
        chunk_jmp['Fecha_anterior'] = chunk_jmp['Fecha'].shift(1)
        chunk_jmp['Delta_tiempo'] = chunk_jmp['Fecha'] - chunk_jmp['Fecha_anterior']
        
        # Crear una máscara para identificar filas consecutivas según la frecuencia esperada
        mask_consecutivo = chunk_jmp['Delta_tiempo'] == pd.to_timedelta(periodos)
        
        # Calcular la diferencia absoluta entre los valores consecutivos
        chunk_jmp['Delta'] = chunk_jmp['Valor'].diff().abs()
        chunk_jmp['Delta'] = chunk_jmp['Delta'].where(mask_consecutivo)

        # Se determina número de salto
        jumpnum = frecuencias['jumpnum']
        
        # Máscara para identificar variaciones mayores al número determinado para cada frecuencia
        mask_variacion = chunk_jmp['Delta'] > jumpnum

        # Se registran los errores en el log
        if mask_variacion.any():
            aligned_mask_salto = mask_variacion.reindex(chunk_jmp.index, fill_value=False)
            for index, row in chunk_jmp[aligned_mask_salto].iterrows():
                self.logger.info('File %s - Row %s - failed val p_jump: %s', archivo, index, row['Valor'])
        else:
            self.logger.info('File: %s - No se encontraron fallos en p_jump', archivo)
      
        # Etiquetado de valores, se inicia con el Estado Anterior
        condicion_0PSO0 = mask_variacion & chunk_jmp['Estado'].isin(['0PSO0', '0PSO1'])
        chunk_jmp.loc[condicion_0PSO0, 'Estado_Anterior'] = chunk_jmp.loc[condicion_0PSO0, 'Estado']
        
        chunk_jmp['Estado'] = chunk_jmp['Estado'].fillna('')
        ## Actualización de estado
        # Condición llenado de 'Estado_Anterior', si aplica
        condicion_0PSO0 = mask_variacion & chunk_jmp['Estado'].isin(['0PSO0', '0PSO1'])
        chunk_jmp.loc[condicion_0PSO0, 'Estado_Anterior'] = chunk_jmp.loc[condicion_0PSO0, 'Estado']

        # Se etiquetan los atípicos
        condicion_0PAT0 = mask_variacion & ((chunk_jmp['Estado'] == '') | chunk_jmp['Estado'].isin(['0PSO0', '0PSO1']))
        chunk_jmp.loc[condicion_0PAT0, 'Estado'] = '0PAT0'

        # Eliminar las columnas temporales antes de devolver el chunk
        chunk_jmp.drop(columns=['Delta', 'Fecha_anterior', 'Delta_tiempo'], axis=1, inplace=True)
        
        # Se copia al chunk original
        chunk.loc[chunk_jmp.index] = chunk_jmp
        return chunk

    def p_valvmin(self, chunk, archivo):
        '''Esta prueba detecta los datos que son máximos y mínimos en horarios distintos a los posibles por la temperatura máxima en el
        día según la radiación solar'''
        # Se crea la columna 'Estado' si no existe
        if 'Estado' not in chunk.columns:
            chunk['Estado'] = ''
        # Se genera la columna 'Estado_anterior' si no existe
        if 'Estado_Anterior' not in chunk.columns:
            chunk['Estado_Anterior'] = ''

        # Se genera filtro para no considerar datos ya catalogados como erróneos
        chunk['Estado'] = chunk['Estado'].fillna('')
        if chunk['Estado'].notna().all(): # Se verifica que no hayan valores nulos en tal columna
            chunk_hm = chunk[~chunk['Estado'].str.startswith('0PER', na=False)].copy()
        else:
            # Si todos los valores son NaN, se copia el chunk
            chunk_hm = chunk.copy()
            
        # Se crean máscaras para los intervalos de tiempo conocidos para valores mínimos
        mask_min_afternoon = (chunk_hm['Fecha'].dt.hour >= 9) & (chunk_hm['Fecha'].dt.hour <= 17)
        # Se filtran los datos para esas horas
        min_validdata = chunk_hm[mask_min_afternoon]

        # Encontrar dos valores mínimos por día
        min_values = chunk_hm.groupby(chunk_hm['Fecha'].dt.date).apply(lambda x: x.nsmallest(1, 'Valor')).reset_index(level=0, drop=True)

        # Verificar los mínimos y obtener las horas correspondientes
        notvalid_min_values = min_values[~min_values.index.isin(min_validdata.index)]

        # Crear una máscara para identificar los índices de los valores no válidos
        notval_min = chunk_hm.index.isin(notvalid_min_values.index) #notval_maxmin = chunk_hm.index.isin(notvalid_values.index)

        # Se registran los errores en el log
        if notval_min.any():
            for index, row in chunk_hm[notval_min].iterrows():
                self.logger.info('File %s - Row %s - failed val p_valvmin: %s', archivo, index, row['Valor'])
        else:
            self.logger.info('File: %s - No se encontraron fallos en p_valvmin', archivo)
        
        chunk_hm['Estado'] = chunk_hm['Estado'].fillna('')
        ## Actualización de estado
        # Condición llenado de 'Estado_Anterior', si aplica
        condicion_0PSO0 = notval_min & chunk_hm['Estado'].isin(['0PSO0', '0PSO1'])
        chunk_hm.loc[condicion_0PSO0, 'Estado_Anterior'] = chunk_hm.loc[condicion_0PSO0, 'Estado']

        # Se etiquetan los atípicos
        condicion_0PAT0 = notval_min & ((chunk_hm['Estado'] == '') | chunk_hm['Estado'].isin(['0PSO0', '0PSO1'])) #notval_maxmin
        chunk_hm.loc[condicion_0PAT0, 'Estado'] = '0PAT0'
        notval_min = notval_min & ~condicion_0PAT0
        
        condicion_0PAT1 = notval_min & (chunk_hm['Estado'] == '0PAT0')
        chunk_hm.loc[condicion_0PAT1, 'Estado'] = '0PAT1'

        # Se copia al chunk original
        chunk.loc[chunk_hm.index] = chunk_hm
        return chunk

    def p_sigmRP(self, chunk, archivo):
        '''Esta prueba calcula, con los datos no etiquetados como erróneos, 4sigmas +- la media -como líms sup e inf- para detectar
        datos atípicos en cada región pluviométrica homogénea obtenida por análisis de componentes principales (Ruíz, Cadena)'''
        # Se crea la columna 'Estado' si no existe
        if 'Estado' not in chunk.columns:
            chunk['Estado'] = ''
        # Se genera la columna 'Estado_anterior' si no existe
        if 'Estado_Anterior' not in chunk.columns:
            chunk['Estado_Anterior'] = ''

        # Se genera filtro para no considerar datos ya catalogados como erróneos
        chunk['Estado'] = chunk['Estado'].fillna('')
        if chunk['Estado'].notna().all(): # Se verifica que no hayan valores nulos en tal columna
            chunk_sgmRP = chunk[~chunk['Estado'].str.startswith('0PER', na=False)].copy()
        else:
            # Si todos los valores son NaN, simplemente copia el chunk
            chunk_sgmRP = chunk.copy()

        # Cargar el archivo de frecuencias
        RefRPLimInf = pd.read_csv('EMAHR_Allinfo_Replcble1.csv', encoding='latin-1', sep=';')
        
        # Se obtienen los valores de umbrales inferiores por estcación
        for station in chunk['Station'].unique():
            Sttn_LimRP =  RefRPLimInf[RefRPLimInf['Station'] == station]['RP_ACP'].values

            # Verificar si Sttn_LimRP es NaN, con el fin de alertar y continuar el análisis
            if np.isnan(Sttn_LimRP).all():
                print(f'No hay valor de LimInfRP para la estación {station}. Se continúa con la siguiente estación')
                continue
            
            Sttn_LimRP = Sttn_LimRP[0]
            LimInfRP = RefRPLimInf.loc[RefRPLimInf['RP_ACP'] == Sttn_LimRP, 'LimInfRP'].iloc[0]

            # Se crea la columna de límites inferiores para agilizar el proceso
            chunk_sgmRP['LimInfRP'] = LimInfRP
            chunk_sgmRP['LimInfRP'] = chunk_sgmRP['LimInfRP'].astype('float64')
            
            # Se comprueba si los datos son inferiores al límite establecido LimInfRP
            mask_out_liminfRP = (chunk_sgmRP['Valor'] < chunk_sgmRP['LimInfRP'])

            # Se registran los errores en el log             
            if mask_out_liminfRP.any():
                aligned_mask_sigmRP = mask_out_liminfRP.reindex(chunk_sgmRP.index, fill_value=False)
                for index, row in chunk_sgmRP[aligned_mask_sigmRP].iterrows():
                    self.logger.info('File %s - Row %s - failed val p_sgmRP: %s', archivo, index, row['Valor'])
            else:
                self.logger.info('File: %s - No se encontraron fallos en p_sgmRP', archivo)
           
            # Actualización de la columna de Estado
            chunk_sgmRP['Estado'] = chunk_sgmRP['Estado'].fillna('')
            # Condición llenado de 'Estado_Anterior', si aplica
            condicion_0PSO0 = mask_out_liminfRP & chunk_sgmRP['Estado'].isin(['0PSO0', '0PSO1'])
            chunk_sgmRP.loc[condicion_0PSO0, 'Estado_Anterior'] = chunk_sgmRP.loc[condicion_0PSO0, 'Estado']
    
            # Se etiquetan los atípicos - '0PAT0'
            condicion_0PAT0 = mask_out_liminfRP & ((chunk_sgmRP['Estado'] == '') | chunk_sgmRP['Estado'].isin(['0PSO0', '0PSO1']))
            chunk_sgmRP.loc[condicion_0PAT0, 'Estado'] = '0PAT0'
            mask_out_liminfRP = mask_out_liminfRP & ~condicion_0PAT0
            # 0PAT1
            condicion_0PAT1 = mask_out_liminfRP & (chunk_sgmRP['Estado'] == '0PAT0')
            chunk_sgmRP.loc[condicion_0PAT1, 'Estado'] = '0PAT1'
            mask_out_liminfRP = mask_out_liminfRP & ~condicion_0PAT1
            # 0PAT2
            condicion_0PAT2 = mask_out_liminfRP & (chunk_sgmRP['Estado'] == '0PAT1')
            chunk_sgmRP.loc[condicion_0PAT2, 'Estado'] = '0PAT2'
            
        # Se copia al chunk original
        chunk.loc[chunk_sgmRP.index] = chunk_sgmRP
        return chunk
        
    def p_sigma(self, chunk, archivo):
        '''Esta prueba calcula, con los datos no etiquetados como erróneos, 4sigmas +- la media -como líms sup e inf- para detectar
        datos atípicos en cada estación'''
        # Se crea la columna 'Estado' si no existe
        if 'Estado' not in chunk.columns:
            chunk['Estado'] = ''
        # Se genera la columna 'Estado_anterior' si no existe
        if 'Estado_Anterior' not in chunk.columns:
            chunk['Estado_Anterior'] = ''

        # Se genera filtro para no considerar datos ya catalogados como erróneos
        chunk['Estado'] = chunk['Estado'].fillna('')
        if chunk['Estado'].notna().all(): # Se verifica que no hayan valores nulos en tal columna
            chunk_sgm = chunk[~chunk['Estado'].str.startswith('0PER', na=False)].copy()
        else:
            # Si todos los valores son NaN, simplemente copia el chunk
            chunk_sgm = chunk.copy()

        # Se calculan los estadísticos para sigma
        mean = chunk_sgm['Valor'].mean()
        std = chunk_sgm['Valor'].std()
        # Con ellos, se establecen los límites superior e inferior
        chunk_sgm['LimSup_Sigma'] = (mean + (4 * std))
        chunk_sgm['LimInf_Sigma'] = (mean - (4 * std))

        # Se etiquetan los valores que sobrepasen el límite
        mask_outbsigma = (chunk_sgm['Valor'] < chunk_sgm['LimInf_Sigma']) | (chunk_sgm['Valor'] > chunk_sgm['LimSup_Sigma'])

        # Se registran los errores en el log
        if mask_outbsigma.any():
            aligned_mask_sigma = mask_outbsigma.reindex(chunk_sgm.index, fill_value=False)
            for index, row in chunk_sgm[aligned_mask_sigma].iterrows():
                self.logger.info('File %s - Row %s - failed val p_sigma: %s', archivo, index, row['Valor'])
        else:
            self.logger.info('File: %s - No se encontraron fallos en p_sigma', archivo)
        
        chunk_sgm['Estado'] = chunk_sgm['Estado'].fillna('')
        ## Actualización de estado
        # Condición llenado de 'Estado_Anterior', si aplica
        condicion_0PSO0 = mask_outbsigma & chunk_sgm['Estado'].isin(['0PSO0', '0PSO1'])
        chunk_sgm.loc[condicion_0PSO0, 'Estado_Anterior'] = chunk_sgm.loc[condicion_0PSO0, 'Estado']

        # Se etiquetan los atípicos - '0PAT0'
        condicion_0PAT0 = mask_outbsigma & ((chunk_sgm['Estado'] == '') | chunk_sgm['Estado'].isin(['0PSO0', '0PSO1']))
        chunk_sgm.loc[condicion_0PAT0, 'Estado'] = '0PAT0'
        mask_outbsigma = mask_outbsigma & ~condicion_0PAT0
        # 0PAT1
        condicion_0PAT1 = mask_outbsigma & (chunk_sgm['Estado'] == '0PAT0')
        chunk_sgm.loc[condicion_0PAT1, 'Estado'] = '0PAT1'
        mask_outbsigma = mask_outbsigma & ~condicion_0PAT1
        # 0PAT2
        condicion_0PAT2 = mask_outbsigma & (chunk_sgm['Estado'] == '0PAT1')
        chunk_sgm.loc[condicion_0PAT2, 'Estado'] = '0PAT2'
        mask_outbsigma = mask_outbsigma & ~condicion_0PAT2
        # 0PAT3
        condicion_0PAT3 = mask_outbsigma & (chunk_sgm['Estado'] == '0PAT2')
        chunk_sgm.loc[condicion_0PAT3, 'Estado'] = '0PAT3'
                    
        # Se eliminan las columnas no deseadas
        if 'LimSup_Sigma' in chunk.columns:
            chunk.drop(columns=['LimSup_Sigma', 'LimInf_Sigma'], axis=1, inplace=True)
    
        chunk.loc[chunk_sgm.index] = chunk_sgm
        return chunk

    def p_coherPNv(self, chunk, archivo):
        '''Esta prueba verifica que un valor tenga coherencia con los 5 valores anteriores y los 5 posteriores
        según su desviación estándar y media'''
        # Se crea la columna 'Estado' si no existe
        if 'Estado' not in chunk.columns:
            chunk['Estado'] = ''    
        # Se genera la columna 'Estado_anterior' si no existe
        if 'Estado_Anterior' not in chunk.columns:
            chunk['Estado_Anterior'] = ''
            
        ## Trabajo con frecuencias
        # Se hace una instancia del método de 'process_freqs' para obtener las frecuencias
        freq_info = self.process_freqs(chunk, archivo)
        if freq_info is None or freq_info['periodos'] is None or freq_info['frecuencias'] is None:
            print(f"No se pudo obtener información de frecuencia para el archivo {archivo}")
            return chunk

        periodos = freq_info['periodos']
        frecuencias = freq_info['frecuencias']

        ## Se asegura la revisión de 5 datos anteriores aún si cambia el chunk
        # Verificar si el archivo ha cambiado
        if self.current_file != archivo:
            # Si el archivo cambió, resetea self.last_rows y actualiza self.current_file
            self.last_rows = None
            self.current_file = archivo
        # Usar self.last_rows para concatenar con el chunk actual
        if self.last_rows is not None:
            chunk = pd.concat([self.last_rows, chunk])
            chunk.reset_index(drop=True)
    
        # Ordenar el chunk por la columna 'Fecha'
        chunk = chunk.sort_values('Fecha').reset_index(drop=True)

        # Asegurarse de que 'periodos' tenga un número antes de la unidad
        if periodos.isalpha():
            periodos = '1' + periodos

        # Crear una columna de diferencia temporal
        chunk['Fecha_anterior'] = chunk['Fecha'].shift(1)
        chunk['Delta_tiempo'] = chunk['Fecha'] - chunk['Fecha_anterior']

        # Se genera filtro para no considerar datos ya catalogados como erróneos o atípicos
        chunk['Estado'] = chunk['Estado'].fillna('')
        if 'Estado' not in chunk.columns or (chunk['Estado'] == '').all():
            chunk_PFvals = chunk.copy()
        else:
            chunk_PFvals = chunk[~chunk['Estado'].str.startswith(('0PER','0PAT'), na=False)].copy()

        # Crear una máscara para identificar filas consecutivas según la frecuencia esperada
        mask_consecutivo = chunk_PFvals['Delta_tiempo'] == pd.to_timedelta(periodos)
        chunk_PFvals['consec_group'] = (~mask_consecutivo).cumsum()

        # Se establecen diferentes ventanas según frecuencias
        windows = {'1T': {'window': 240}, '5T': {'window': 72}, '10T': {'window': 48}, '1H': {'window': 11}}
        window_size = windows[periodos]['window']
        half_window = window_size // 2

        # Filtrar grupos que tienen al menos el tamaño de ventana necesario
        group_counts = chunk_PFvals['consec_group'].value_counts()
        valid_groups = group_counts[group_counts >= window_size].index
        chnk_cohPFvl = chunk_PFvals[chunk_PFvals['consec_group'].isin(valid_groups)]

        # Verificar que los datos anteriores y posteriores sean consecutivos
        valid_indices = []
        for i in range(half_window, len(chnk_cohPFvl) - half_window):
            if all(mask_consecutivo[i-half_window:i+half_window]):
                valid_indices.append(chnk_cohPFvl.index[i])

        chnk_cohPFvl = chnk_cohPFvl.loc[valid_indices]

        # Calcular el promedio y desviación estándar de los registros anteriores y posteriores
        chnk_cohPFvl['mean_PF'] = chnk_cohPFvl['Valor'].rolling(window=window_size, center=True).mean()
        chnk_cohPFvl['std_PF'] = chnk_cohPFvl['Valor'].rolling(window=window_size, center=True).std()

        # Calcular los límites superior e inferior
        chnk_cohPFvl['lim_inf'] = chnk_cohPFvl['mean_PF'] - (3 * chnk_cohPFvl['std_PF'])
        chnk_cohPFvl['lim_sup'] = chnk_cohPFvl['mean_PF'] + (3 * chnk_cohPFvl['std_PF'])

        # Máscara para identificar valores fuera de los límites
        mask_varPF = (chnk_cohPFvl['Valor'] < chnk_cohPFvl['lim_inf']) | (chnk_cohPFvl['Valor'] > chnk_cohPFvl['lim_sup'])

        # Se registran los errores en el log
        if mask_varPF.any():
            aligned_mask_sigma = mask_varPF.reindex(chnk_cohPFvl.index, fill_value=False)
            for index, row in chnk_cohPFvl[aligned_mask_sigma].iterrows():
                self.logger.info('File %s - Row %s - failed val p_coherPNv %s', archivo, index, row['Valor'])
        else:
            self.logger.info('File: %s - No se encontraron fallos en p_coherPNv', archivo)
        
        chnk_cohPFvl['Estado'] = chnk_cohPFvl['Estado'].fillna('')
        # Lógica de etiquetado para 'Estado', sospechoso, '0PSO0'
        condicion_0PSO0 = mask_varPF & ((chnk_cohPFvl['Estado'] == ''))
        chnk_cohPFvl.loc[condicion_0PSO0, 'Estado'] = '0PSO0'
        mask_varPF = mask_varPF & ~condicion_0PSO0
        # 0PSO1
        condicion_0PSO1 = mask_varPF & (chnk_cohPFvl['Estado'] == '0PSO0')
        chnk_cohPFvl.loc[condicion_0PSO1, 'Estado'] = '0PSO1'
        mask_varPF = mask_varPF & ~condicion_0PSO1
        # 0PSO2
        condicion_0PSO2 = mask_varPF & (chnk_cohPFvl['Estado'] == '0PSO1')
        chnk_cohPFvl.loc[condicion_0PSO2, 'Estado'] = '0PSO2'
    
        # Se asegura la verificación de los valores anteriores si hubo cambio de chunk
        self.last_rows = chnk_cohPFvl.tail(5)

        # Eliminar las columnas temporales antes de devolver el chunk
        if 'Fecha_anterior' in chunk.columns:
            chunk.drop(columns=['Fecha_anterior','Delta_tiempo'], axis=1, inplace=True)
            
        # Copiar datos de chunk_coher al chunk original
        chunk.loc[chnk_cohPFvl.index] = chnk_cohPFvl
        # Continuar eliminando filas
        if 'mean_5' in chunk.columns:
            chunk.drop(columns=['mean_5','std_5','lim_inf','lim_sup'], axis=1, inplace=True)
        
        return chunk

    def p_varmin(self, chunk, archivo):
        '''Esta prueba detecta los valores horarios que no varían en 1.0 % durante la hora'''
        # Se crea la columna 'Estado' si no existe
        if 'Estado' not in chunk.columns:
            chunk['Estado'] = ''
        # Se genera la columna 'Estado_Anterior' si no existe
        if 'Estado_Anterior' not in chunk.columns:
            chunk['Estado_Anterior'] = ''
        ## Trabajo con frecuencias
        # Se hace una instancia del método de 'process_freqs' para obtener las frecuencias
        freq_info = self.process_freqs(chunk, archivo)
        if freq_info is None or freq_info['periodos'] is None or freq_info['frecuencias'] is None:
            print(f"No se pudo obtener información de frecuencia para el archivo {archivo}")
            return chunk

        periodos = freq_info['periodos']
        frecuencias = freq_info['frecuencias']
    
        # Asegurarse de que 'periodos' tenga un número antes de la unidad
        if periodos.isalpha():
            periodos = '1' + periodos
    
        # Ordenar el chunk por la columna 'Fecha'
        chunk = chunk.sort_values('Fecha').reset_index(drop=True)

        # Se genera filtro para no considerar datos ya catalogados como erróneos o atípicos
        chunk['Estado'] = chunk['Estado'].fillna('')
        if 'Estado' in chunk.columns and chunk['Estado'].notna().all():
            chunk_varhmn = chunk[~chunk['Estado'].str.startswith(('0PER', '0PAT'), na=False)].copy()
        else:
            chunk_varhmn = chunk.copy()

        # Crear una columna de diferencia temporal y otras columnas temporales
        chunk_varhmn['Fecha_anterior'] = chunk_varhmn['Fecha'].shift(1)
        chunk_varhmn['Delta_tiempo'] = chunk_varhmn['Fecha'] - chunk_varhmn['Fecha_anterior']
    
        # Crear una máscara para identificar filas consecutivas según la frecuencia esperada
        freq_map = {'H': '1H', 'T': '1T', '5T': '5T', '10T': '10T'}
        expected_delta = pd.to_timedelta(freq_map.get(periodos, '1H'))
        mask_consecut = chunk_varhmn['Delta_tiempo'] == expected_delta
    
        # Calcular la diferencia absoluta entre los valores consecutivos
        chunk_varhmn['Delta'] = chunk_varhmn['Valor'].diff().abs()
        chunk_varhmn['Delta'] = chunk_varhmn['Delta'].where(mask_consecut)
    
        # Aplicar la máscara de las horas soleadas después de crear las columnas temporales
        mask_sunny2 = (chunk_varhmn['Fecha'].dt.hour >= 6) & (chunk_varhmn['Fecha'].dt.hour <= 18)
        mask_sun2 = chunk_varhmn[mask_sunny2].copy()
    
        if periodos == 'H':
            # Máscara para identificar variaciones menores a 0.1 o 0.01
            mask_varhmin = mask_sun2['Delta'] < 0.1
        else:
            # Agrupar por horas y verificar si alguna variación dentro de la hora excede 1.0
            mask_sun2.loc[:, 'Fecha_hora'] = mask_sun2['Fecha'].dt.floor('h')
            hora_groups = mask_sun2.groupby('Fecha_hora')
            mask_varhmin = hora_groups['Delta'].transform(lambda x: (x < 0.01).any())

        # Se registran los errores en el log
        if mask_varhmin.any():
            aligned_mask_varmin = mask_varhmin.reindex(mask_sun2.index, fill_value=False)
            for index, row in mask_sun2[aligned_mask_varmin].iterrows():
                self.logger.info('File %s - Row %s - failed val p_varmin: %s', archivo, index, row['Valor'])
        else:
            self.logger.info('File: %s - No se encontraron fallos en p_varmin', archivo)
        
        mask_sun2.loc[:, 'Estado'] = mask_sun2['Estado'].fillna('')
        # Lógica de etiquetado para 'Estado', sospechoso, '0PSO0'
        condicion_0PSO0 = mask_varhmin & ((mask_sun2['Estado'] == ''))
        mask_sun2.loc[condicion_0PSO0, 'Estado'] = '0PSO0'
        mask_varhmin = mask_varhmin & ~condicion_0PSO0
        # 0PSO1
        condicion_0PSO1 = mask_varhmin & (mask_sun2['Estado'] == '0PSO0')
        mask_sun2.loc[condicion_0PSO1, 'Estado'] = '0PSO1'
        mask_varhmin = mask_varhmin & ~condicion_0PSO1
        # 0PSO2
        condicion_0PSO2 = mask_varhmin & (mask_sun2['Estado'] == '0PSO1')
        mask_sun2.loc[condicion_0PSO2, 'Estado'] = '0PSO2'
        mask_varhmin = mask_varhmin & ~condicion_0PSO2
        # 0PSO2
        condicion_0PSO3 = mask_varhmin & (mask_sun2['Estado'] == '0PSO2')
        mask_sun2.loc[condicion_0PSO3, 'Estado'] = '0PSO3'
    
        # Copiar datos de chunk_jmp al chunk original
        chunk.loc[mask_sun2.index] = mask_sun2
    
        # Eliminar las columnas temporales antes de devolver el chunk
        columns_to_drop = ['Delta', 'Fecha_anterior', 'Delta_tiempo', 'Fecha_hora']
        columns_to_drop = [col for col in columns_to_drop if col in chunk.columns]
        chunk.drop(columns=columns_to_drop, axis=1, inplace=True)
    
        return chunk

    def procesar_archivos(self, funcion_evaluacion):
        '''Este método procesa la lectura y guardado de los archivos para todas las pruebas'''
        archivos = self.ruta_archivos

        archivos_salida = []  # Lista para almacenar nombres de archivos de salida

        # Se recorre cada archivo en la carpeta
        for archivo in archivos:
            if archivo.endswith('.csv'):
                ruta_archivo = os.path.join(self.dir_files, archivo)

                reader = pd.read_csv(ruta_archivo, encoding='latin-1', chunksize=self.chunk_size)#,dtype={7: 'str'}, low_memory=False)
                resultados = []

                for chunk in reader:
                    try:
                        chunk['Fecha'] = pd.to_datetime(chunk['Fecha'], format='%Y-%m-%d %H:%M:%S.%f')
                    except ValueError:
                        chunk['Fecha'] = pd.to_datetime(chunk['Fecha'], format='%Y-%m-%d %H:%M:%S')

                    chunk['Station'] = chunk['Station'].astype('int64')

                    # try:
                    #     chunk_resultado, _ = funcion_evaluacion(chunk, archivo)  # Desempaqueta solo el DataFrame
                    # except ValueError:
                    chunk_resultado = funcion_evaluacion(chunk, archivo)  # Desempaqueta solo el DataFrame
                    resultados.append(chunk_resultado)

                if not resultados:  # Se verifica si la lista está vacía
                    self.logger.warning('No hay resultados válidos para concatenar en el archivo %s. Continuando con el siguiente.', archivo)
                    continue
                    
                resultados_consolidados = pd.concat(resultados)

                # Genera el nombre del archivo de salida conservando los primeros 19 caracteres del nombre del archivo original
                nombre_archivo_salida = archivo[:19] + '_qc.csv'

                resultados_consolidados.to_csv(os.path.join(self.dir_files, nombre_archivo_salida), encoding='latin-1', index=False)

                archivos_salida.append(nombre_archivo_salida)  # Agregar el nombre del archivo a la lista
            
        # Actualiza self.ruta_archivos para que la próxima prueba procese los resultados de esta prueba
        self.ruta_archivos = archivos_salida
        # Se fija el log de procesamiento completo de archivos
        self.logger.info('Procesamiento completo de archivos de estaciones HR. Archivos generados: %s', archivos_salida)

In [79]:
procesador = AutomatTS10cmEMA('RawUnmodified_TS-10cm') #Test_QC/T

In [ ]:
procesador.procesar_archivos(procesador.p_transm)

In [15]:
procesador.procesar_archivos(procesador.p_estruc)

C:\Users\palvarez\AppData\Local\Temp\ipykernel_13452\3458943399.py:795: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in reader:


In [16]:
procesador.procesar_archivos(procesador.p_limrig)

C:\Users\palvarez\AppData\Local\Temp\ipykernel_13452\3458943399.py:795: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in reader:


In [71]:
procesador.procesar_archivos(procesador.p_perst)

In [81]:
## Analizar presencia de '0PSO0'
# Cambia este directorio al lugar donde tengas tu carpeta 'ReadytoCassandraFiles'
directorio = 'RawUnmodified_TS-10cm' #r'Test_QC/10T'
# Lista para almacenar los nombres de los archivos que contienen '0PSO1' en la columna 'Estado'
cantdatos_0PER = []

# Itera sobre cada archivo en el directorio
for archivo in os.listdir(directorio):
    if archivo.endswith('_qc.csv'):
        # Construye la ruta completa al archivo
        ruta_archivo = os.path.join(directorio, archivo)
        # Especifica los tipos de dato para las columnas deseadas
        tipos_de_dato = {'Estado': str, 'Estado_Anterior': str}
        # Lee el archivo CSV en un DataFrame de pandas
        df = pd.read_csv(ruta_archivo, encoding='latin-1',dtype=tipos_de_dato)
        # Checa cuántos '0PER0' hay en la columna 'Estado'
        count_0PER0 = df['Estado'].value_counts().get('0PER0', 0)
        # Guarda el nombre del archivo y la cantidad de datos con '0PER0'
        cantdatos_0PER.append((archivo, count_0PER0))

# Imprime la cantidad de datos con '0PER0' en cada archivo
#for archivo, count in cantdatos_0PER:
    #print(f"Archivo: {archivo}, Datos con '0PER0': {count}")

# Crear un DataFrame a partir de los resultados
df_resultados = pd.DataFrame(cantdatos_0PER, columns=['Archivo', 'Cantidad_0PER0'])
# Guardar resultados
prueba = 'perst'
df_resultados.to_csv(f'cant_{prueba}_8.csv')

In [ ]:
procesador.procesar_archivos(procesador.p_jump)

In [ ]:
procesador.procesar_archivos(procesador.p_valvmin)

In [ ]:
procesador.procesar_archivos(procesador.p_sigmRP)

In [ ]:
procesador.procesar_archivos(procesador.p_sigma)

In [ ]:
procesador.procesar_archivos(procesador.p_coherPNv)

In [ ]:
procesador.procesar_archivos(procesador.p_varmin)